In [2]:
import time
import lattice_challenges_01

# in this notebook's comments, "triangular matrix"
#   should always have the meaning of "lower triangular matrix"

In [8]:
# in triangular matrix, this function changes row (vector)
#   at index line_index in input_matrix by subtracting row
#   at index by_index k times
def sub_in_tr(input_matrix, line_index, by_index, k):
    for i in range(by_index+1):
        input_matrix[line_index, i] -= k*input_matrix[by_index, i]

# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; the orthogonalized vector is v_i;
#   the vector by which it is improved is v_j
def gram_schmidt_mu(v_i, v_j):
    return (v_i*v_j)/(v_j*v_j)

# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; here, basis is the matrix of row-vectors
#   being orthogonalized and gso is the list (or matrix) of already
#   orthogonalized vectors
def mu(i, j, basis, gso):
    return gram_schmidt_mu(basis[i], gso[j])

# this function returns a matrix which is the result
#   of Gram-Shmidt orthogonalization process (not orthonormalization)
def gram_schmidt_orthogonalization(basis):
    ret = matrix(RR, basis.nrows(), basis.ncols())
    for i in range(basis.nrows()):
        v = basis[i]
        for j in range(i):
            v -= gram_schmidt_mu(v, ret[j])*ret[j]
        ret[i] = v
    return ret

# test used in classical LLL
def classical_Lovasz_condition(b, g, i, delta):
    return (delta - mu(i, i-1, b, g)**2)*(g[i-1]*g[i-1]) <= g[i]*g[i]

# used in LLL (regardless of which condition is used)
def swap_rows_in_LLL(b, g, i):
    g_i_1 = g[i] + mu(i, i-1, b, g)*g[i-1]
    g[i] = g[i-1] - gram_schmidt_mu(b[i-1], g_i_1)*g_i_1
    g[i-1] = g_i_1
    b[i], b[i-1] = b[i-1], b[i]

# implementation of classical LLL algorithm
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (including 0.75)
def LLL_usual_condition(input_basis, delta):
    basis = copy(input_basis)
    n = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and not classical_Lovasz_condition(basis, gso, i, delta):
            swap_rows_in_LLL(basis, gso, i)
            i -= 1
        else:
            i += 1
    return basis

# implementation of LLL algorithm with different condition as used in
#   https://link.springer.com/chapter/10.1007/978-3-319-94821-8_10
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (not including 0.75)
def LLL_unusual_condition(input_basis, delta):
    basis = copy(input_basis)
    n = basis.nrows()
    gso = gram_schmidt_orthogonalization(basis)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            basis[i] -= round(mu(i, j, basis, gso))*basis[j]
        if i > 0 and delta*(gso[i-1]*gso[i-1]) > gso[i]*gso[i]:
            swap_rows_in_LLL(basis, gso, i)
            i -= 1
        else:
            i += 1
    return basis

# auxiliary function used in triangularize_basis
def add_to_tr_diag(ret, cur_row, ort_basis, diag_value):
    for i in range(cur_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[cur_row], ort_basis[i])
        ort_basis[cur_row] -= t*ort_basis[i]
        ret[cur_row, i] = t*ret[i, i]
    ret[cur_row, cur_row] = diag_value

# auxiliary function used in triangularize_basis
def add_to_tr(ret, cur_row, ort_basis):
    for i in range(cur_row-1, -1, -1):
        t = gram_schmidt_mu(ort_basis[cur_row], ort_basis[i])
        ort_basis[cur_row] -= t*ort_basis[i]
        ret[cur_row, i] = t*ret[i, i]
    ret[cur_row, cur_row] = sqrt(ort_basis[cur_row]*ort_basis[cur_row])

# function returning a triangular matrix which has the same Gramian matrix
#   as the input b
# if n_of_small_diag is zero,
#   it does not change the order of rows in b
# if n_of_small_diag is positive integer,
#   the ordering of rows in input b is changed (side effect)
#   and the ordering of first n_of_small_diag
#   rows is chosen (gradually, greedily) so that the corresponding
#   diagonal elements in the resulting matrix are the lowest possible
def triangularize_basis(b, n_of_small_diag=0):
    ret = matrix(RR, b.nrows(), b.nrows())
    ort_b = matrix(RR, b)
    c = 0  # current row index
    while c < n_of_small_diag:
        min_value = math.inf  # using infinity from math module
        min_index = -1
        for m in range(c, ort_b.nrows()):
            v = ort_b[m]
            for i in range(c):
                v -= gram_schmidt_mu(v, ort_b[i])*ort_b[i]
            t = sqrt(v*v)
            if t < min_value:
                min_value = t
                min_index = m
        if min_index != c:
            ort_b[c], ort_b[min_index] = ort_b[min_index], ort_b[c]
            b[c], b[min_index] = b[min_index], b[c]
        add_to_tr_diag(ret, c, ort_b, min_value)
        c += 1
    while c < ort_b.nrows():
        add_to_tr(ret, c, ort_b)
        c += 1
    return ret

# this function reduces the input tr_basis from bottom
# performs the same changes on parallel_basis
# the function has side effect on both input parameters
# the return value indicates whether any change of the input
#   parameters has been made (is True if so)
def reduce_triangular_basis(tr_basis, parallel_basis):
    change = False
    for c in range(tr_basis.nrows()-1, -1, -1):
        for m in range(c+1, tr_basis.nrows()):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                sub_in_tr(tr_basis, m, c, t)
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# probably has exactly the same effect as reduce_triangular_basis above
def reduce_triangular_basis_from_top(tr_basis, parallel_basis):
    change = False
    for m in range(1, tr_basis.nrows()):
        for c in range(m-1, -1, -1):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                sub_in_tr(tr_basis, m, c, t)
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# obsolete function for reduction (does not terminate in some cases)
# reduces the basis using reduce_triangular_basis as long
#   as anything in the basis changes
# uses copy of input_basis (does not have side effects)
def reduce_basis_using_triangular(input_basis, n_of_small_diag=0):
    basis = copy(input_basis)
    change = True
    while change:
        tr_basis = triangularize_basis(basis, n_of_small_diag)
        change = reduce_triangular_basis(tr_basis, basis)
    return basis

# returns the sum of natural logarithms of the euclidean norms
#   of vectors of input_matrix
def euclidean_sum_of_logarithms_of_norms(input_matrix):
    sum_of_logarithms_of_squared_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = input_matrix[i]*input_matrix[i]
        sum_of_logarithms_of_squared_norms += float(log(t))
    return sum_of_logarithms_of_squared_norms/2

# returns the sum of natural logarithms of the elements of the input alist
def sum_of_logarithms(alist):
    ret = 0.0
    for val in alist:
        ret += float(log(val))
    return ret

# sorts input b via side effect; returns a list of squared euclidean norms
def sort_by_norms(b):
    squared_norms = [(b[i]*b[i], i) for i in range(b.nrows())]
    squared_norms.sort()
    perm_dict = {}
    for i in range(len(squared_norms)):
        perm_dict[i] = squared_norms[i][1]
    while perm_dict:
        start_index, next_index = perm_dict.popitem()
        start_row = b[start_index]
        cur_index = start_index
        while next_index != start_index:
            b[cur_index] = b[next_index]
            cur_index = next_index
            next_index = perm_dict.pop(next_index)
        b[cur_index] = start_row
    return [squared_norms[i][0] for i in range(b.nrows())]

# sort by euclidean norms and reduce as long
#   as sum of logarithms gets lower
# works with a copy of input_basis; does not have side effects
def reduce_basis_using_triangular_new(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        tr_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(tr_basis, basis):
            return basis
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# performs changes in triangular matrix (tr) corresponding to swaping
#   rows at indices i and i-1 in the corresponding basis, so that
#   gramian matrices of tr and the corresponding basis remain the same
def swap_rows_in_tr(tr, i):
    bk = tr[i, i-1]
    ak = tr[i-1, i-1]
    bk1 = tr[i, i]
    tr[i, i-1] = sqrt(bk**2+bk1**2)
    tr[i-1, i-1] = ak*bk/tr[i, i-1]
    tr[i-1, i] = ak*bk1/tr[i, i-1]
    tr[i, i] = 0
    tr[i], tr[i-1] = tr[i-1], tr[i]
    for x in range(i+1, tr.nrows()):
        xa = tr[x, i-1]
        tr[x, i-1] = (xa*bk+tr[x, i]*bk1)/tr[i-1, i-1]
        tr[x, i] = (xa*ak-tr[x, i-1]*tr[i, i-1])/tr[i, i]

# faster implementation of classical LLL
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_usual_condition_using_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * \
                   (delta-(tr[i, i-1]/tr[i-1, i-1])**2) > tr[i, i]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

# faster implementation of LLL with different condition
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_unusual_condition_using_triangular(input_basis, delta):
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis, 0)
    i = 0
    flag = True
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]*sqrt_delta > tr[i, i]:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis


In [9]:
def taxicab_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t += abs(input_matrix[i, j])
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm,
            sum_of_norms, sum_of_logarithms_of_norms)

def chebyshev_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0
    sum_of_norms = 0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0
        for j in range(input_matrix.ncols()):
            t2 = abs(input_matrix[i, j])
            if t2 > t:
                t = t2
        sum_of_norms += t
        sum_of_logarithms_of_norms += float(log(t))
        if t > longest_vector_norm:
            longest_vector_norm = t
        if t < shortest_vector_norm:
            shortest_vector_norm = t
    return (shortest_vector_norm, longest_vector_norm,
            sum_of_norms, sum_of_logarithms_of_norms)

def euclidean_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0.0
    sum_of_norms = 0.0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0.0
        for j in range(input_matrix.ncols()):
            t += input_matrix[i, j]**2
        r = sqrt(t)
        sum_of_norms += r
        sum_of_logarithms_of_norms += float(log(r))
        if r > longest_vector_norm:
            longest_vector_norm = r
        if r < shortest_vector_norm:
            shortest_vector_norm = r
    return (shortest_vector_norm, longest_vector_norm,
            sum_of_norms, sum_of_logarithms_of_norms)

def all_properties(input_matrix):
    print(euclidean_properties(input_matrix))
    print(chebyshev_properties(input_matrix))
    print(taxicab_properties(input_matrix))


In [10]:
# assumed the inputs are integer matrix and integer vector
#   of the same dimension (row length)
def check_vector_in_lattice(basis, a_vector):
    return basis.solve_left(a_vector).denominator() == 1

# assumed the inputs are integer matrices of the same dimension
def check_reduction_correctness(original_basis, tested_matrix):
    ret = True
    for a_row in tested_matrix:
        if not check_vector_in_lattice(original_basis, a_row):
            return False
    return original_basis.rank()==tested_matrix.rank()


In [11]:
def add_matrix_to_file(filename, matrixname, the_matrix):
    f = open(filename, 'a')
    f.write("\ndef "+str(matrixname)+"():\n    return [")
    for i in range(the_matrix.nrows()-1):
        f.write(str(list(the_matrix[i]))+",\n            ")
    if the_matrix.nrows()>0:
        f.write(str(list(the_matrix[the_matrix.nrows()-1])))
    f.write("]\n")
    f.close()
